In [2]:
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim import matutils, models
import scipy.sparse

In [3]:
data = pd.read_pickle('all_data.pkl')

In [7]:
add_stop_words=['app','use','good','just','google','like','work',
                'time','need','dont','make','phone','update',
                'great','im']
stop_words1=text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Topic Modeling - Attempt #1 (All Text)

-  Count- Vectorizer

In [5]:
stop_words1=text.ENGLISH_STOP_WORDS.union(add_stop_words)
cv1=CountVectorizer(stop_words=stop_words1, token_pattern="\\b[a-z][a-z]+\\b",
                    min_df=3,max_df=0.85, max_features=10000, ngram_range=(1,2))
df_cv1 = cv1.fit_transform(data.content)
df_review1=pd.DataFrame(df_cv1.toarray(), index= [data.title], columns=cv1.get_feature_names())

- TfidfVectorizer

In [6]:
stop_words1=text.ENGLISH_STOP_WORDS.union(add_stop_words)
cv2=TfidfVectorizer(stop_words=stop_words1, token_pattern="\\b[a-z][a-z]+\\b",
                    min_df=3,max_df=0.85, max_features=10000, ngram_range=(1,2))
df_cv2 = cv2.fit_transform(data.content)
df_review2=pd.DataFrame(df_cv2.toarray(), index= [data.title], columns=cv2.get_feature_names())

In [7]:
tdm1 = df_review1.transpose()
tdm2 = df_review2.transpose()

-  put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus

In [8]:
sparse_counts1 = scipy.sparse.csr_matrix(tdm1)
corpus1 = matutils.Sparse2Corpus(sparse_counts1)

sparse_counts2 = scipy.sparse.csr_matrix(tdm2)
corpus2 = matutils.Sparse2Corpus(sparse_counts2)

- Gensim also requires dictionary of the all terms and their respective location in the term-document matrix

In [9]:
id2word1 = dict((v, k) for k, v in cv1.vocabulary_.items())
id2word2 = dict((v, k) for k, v in cv2.vocabulary_.items())

In [10]:
num_topic=5

In [11]:
print('Count-Vectorizer: num_topic= {}'.format(num_topic))
lda = models.LdaModel(corpus=corpus1, id2word=id2word1, num_topics=num_topic, passes=5)
for i in range(len(lda.print_topics())):
    print(lda.print_topics()[i])

Count-Vectorizer: num_topic= 5
(0, '0.022*"tv" + 0.015*"play" + 0.015*"download" + 0.012*"want" + 0.011*"ad" + 0.011*"video" + 0.010*"know" + 0.010*"watch" + 0.010*"thing" + 0.008*"music"')
(1, '0.032*"nice" + 0.028*"love" + 0.020*"meet" + 0.017*"best" + 0.014*"really" + 0.014*"thank" + 0.013*"application" + 0.012*"ok" + 0.012*"star" + 0.011*"easy"')
(2, '0.019*"map" + 0.017*"try" + 0.016*"fix" + 0.014*"issue" + 0.014*"device" + 0.013*"problem" + 0.013*"doesnt" + 0.012*"connect" + 0.012*"android" + 0.009*"crash"')
(3, '0.030*"screen" + 0.024*"photo" + 0.020*"wallpaper" + 0.015*"button" + 0.012*"option" + 0.012*"search" + 0.011*"doesnt" + 0.011*"picture" + 0.010*"selfie" + 0.010*"version"')
(4, '0.030*"feature" + 0.028*"add" + 0.019*"art" + 0.018*"option" + 0.014*"task" + 0.012*"better" + 0.010*"user" + 0.009*"sound" + 0.008*"version" + 0.007*"really"')


In [12]:
print('Tfidf-Vectorizer: num_topic= {}'.format(num_topic))
lda = models.LdaModel(corpus=corpus2, id2word=id2word2, num_topics=num_topic, passes=5)
for i in range(len(lda.print_topics())):
    print(lda.print_topics()[i])

Tfidf-Vectorizer: num_topic= 5
(0, '0.082*"nice" + 0.029*"love" + 0.021*"bluetooth" + 0.015*"useful" + 0.015*"cool" + 0.013*"vr" + 0.011*"gallery" + 0.010*"easy" + 0.009*"connect" + 0.008*"star"')
(1, '0.014*"art" + 0.012*"wallpaper" + 0.010*"crash" + 0.010*"photo" + 0.008*"best" + 0.007*"selfie" + 0.007*"version" + 0.007*"open" + 0.007*"try" + 0.006*"load"')
(2, '0.014*"ok" + 0.009*"feature" + 0.009*"add" + 0.007*"meet" + 0.007*"option" + 0.007*"remote" + 0.006*"helpful" + 0.006*"thank" + 0.006*"measure" + 0.006*"screen"')
(3, '0.016*"map" + 0.013*"bad" + 0.012*"task" + 0.009*"amaze" + 0.009*"headphone" + 0.009*"apps" + 0.008*"really" + 0.008*"doesnt" + 0.007*"accurate" + 0.007*"measurement"')
(4, '0.024*"tv" + 0.016*"awesome" + 0.011*"play" + 0.009*"learn" + 0.009*"podcast" + 0.009*"keyboard" + 0.008*"android tv" + 0.008*"class" + 0.008*"download" + 0.008*"youtube"')


# Topic Modeling - Attempt #2 (Nouns Only)

In [8]:
data_nouns = pd.read_pickle('data_nouns.pkl')

-  Count- Vectorizer

In [9]:
cv1_2=CountVectorizer(stop_words=stop_words1, token_pattern="\\b[a-z][a-z]+\\b",
                    min_df=3,max_df=0.85, max_features=10000, ngram_range=(1,2))
df_cv1_2 = cv1_2.fit_transform(data_nouns.content)
df_review1_2=pd.DataFrame(df_cv1_2.toarray(), index= [data.title], columns=cv1_2.get_feature_names())

- TfidfVectorizer

In [10]:
cv2_2=TfidfVectorizer(stop_words=stop_words1, token_pattern="\\b[a-z][a-z]+\\b",
                    min_df=3,max_df=0.85, max_features=10000, ngram_range=(1,2))
df_cv2_2 = cv2_2.fit_transform(data_nouns.content)
df_review2_2=pd.DataFrame(df_cv2_2.toarray(), index= [data.title], columns=cv2_2.get_feature_names())

In [17]:
# Create the gensim corpus
corpus1_2 = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(df_review1_2.transpose()))
corpus2_2 = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(df_review2_2.transpose()))
# Create the vocabulary dictionary
id2word1_2 = dict((v, k) for k, v in cv1_2.vocabulary_.items())
id2word2_2 = dict((v, k) for k, v in cv2_2.vocabulary_.items())

In [18]:
num_topic=5

In [19]:
print('Count-Vectorizer: num_topic= {}'.format(num_topic))
lda = models.LdaModel(corpus=corpus1_2, id2word=id2word1_2, num_topics=num_topic, passes=5)
for i in range(len(lda.print_topics())):
    print(lda.print_topics()[i])

Count-Vectorizer: num_topic= 5
(0, '0.035*"tv" + 0.031*"option" + 0.026*"wallpaper" + 0.020*"apps" + 0.015*"feature" + 0.014*"mode" + 0.014*"download" + 0.014*"thing" + 0.012*"selfie" + 0.012*"image"')
(1, '0.027*"ad" + 0.027*"video" + 0.026*"day" + 0.018*"music" + 0.016*"podcast" + 0.015*"play" + 0.015*"data" + 0.014*"view" + 0.013*"youtube" + 0.011*"analytics"')
(2, '0.036*"version" + 0.019*"meet" + 0.016*"star" + 0.015*"way" + 0.014*"device" + 0.014*"file" + 0.014*"picture" + 0.014*"thank" + 0.013*"page" + 0.012*"crash"')
(3, '0.042*"feature" + 0.036*"photo" + 0.030*"art" + 0.026*"task" + 0.016*"application" + 0.016*"option" + 0.014*"share" + 0.014*"account" + 0.011*"access" + 0.010*"desktop"')
(4, '0.040*"screen" + 0.032*"map" + 0.022*"button" + 0.020*"problem" + 0.016*"home" + 0.016*"issue" + 0.013*"sound" + 0.012*"headphone" + 0.011*"location" + 0.010*"doesnt"')


In [21]:
print('Tfidf-Vectorizer: num_topic= {}'.format(num_topic))
lda = models.LdaModel(corpus=corpus2_2, id2word=id2word2_2, num_topics=num_topic, passes=5)
for i in range(len(lda.print_topics())):
    print(lda.print_topics()[i])

Tfidf-Vectorizer: num_topic= 5
(0, '0.026*"photo" + 0.018*"feature" + 0.018*"meet" + 0.018*"task" + 0.015*"file" + 0.009*"watch" + 0.009*"gallery" + 0.009*"lot" + 0.009*"download" + 0.008*"pdf"')
(1, '0.024*"map" + 0.018*"version" + 0.013*"selfie" + 0.011*"doesnt" + 0.011*"measurement" + 0.009*"measure" + 0.009*"problem" + 0.008*"vr" + 0.008*"fun" + 0.008*"idea"')
(2, '0.019*"ok" + 0.017*"headphone" + 0.013*"thank" + 0.012*"volume" + 0.010*"account" + 0.010*"class" + 0.010*"device" + 0.009*"desktop" + 0.009*"culture" + 0.009*"way"')
(3, '0.032*"art" + 0.014*"screen" + 0.013*"sound" + 0.012*"ad" + 0.012*"option" + 0.012*"picture" + 0.011*"application" + 0.010*"feature" + 0.010*"experience" + 0.010*"home"')
(4, '0.032*"wallpaper" + 0.028*"tv" + 0.016*"crash" + 0.014*"apps" + 0.012*"video" + 0.010*"data" + 0.010*"play" + 0.009*"music" + 0.009*"quality" + 0.008*"game"')


# Topic Modeling - Attempt #3 (Nouns + Adj + Verb Only)

In [22]:
data_nouns_adj_v = pd.read_pickle('data_nouns_adj_v.pkl')

-  Count- Vectorizer

In [23]:
cv1_3=CountVectorizer(stop_words=stop_words1, token_pattern="\\b[a-z][a-z]+\\b",
                    min_df=3,max_df=0.85, max_features=10000, ngram_range=(1,2))
df_cv1_3 = cv1_3.fit_transform(data_nouns_adj_v.content)
df_review1_3=pd.DataFrame(df_cv1_3.toarray(), index= [data.title], columns=cv1_3.get_feature_names())

- TfidfVectorizer

In [24]:
cv2_3=TfidfVectorizer(stop_words=stop_words1, token_pattern="\\b[a-z][a-z]+\\b",
                    min_df=3,max_df=0.85, max_features=10000, ngram_range=(1,2))
df_cv2_3 = cv2_3.fit_transform(data_nouns.content)
df_review2_3=pd.DataFrame(df_cv2_3.toarray(), index= [data.title], columns=cv2_3.get_feature_names())

In [25]:
# Create the gensim corpus
corpus1_3 = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(df_review1_3.transpose()))
corpus2_3 = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(df_review2_3.transpose()))
# Create the vocabulary dictionary
id2word1_3 = dict((v, k) for k, v in cv1_3.vocabulary_.items())
id2word2_3 = dict((v, k) for k, v in cv2_3.vocabulary_.items())

In [26]:
num_topic=5

In [27]:
print('Count-Vectorizer: num_topic= {}'.format(num_topic))
lda = models.LdaModel(corpus=corpus1_3, id2word=id2word1_3, num_topics=num_topic, passes=5)
for i in range(len(lda.print_topics())):
    print(lda.print_topics()[i])

Count-Vectorizer: num_topic= 5
(0, '0.011*"want" + 0.010*"way" + 0.010*"watch" + 0.010*"screen" + 0.010*"set" + 0.009*"account" + 0.009*"doesnt" + 0.009*"page" + 0.007*"desktop" + 0.007*"device"')
(1, '0.022*"love" + 0.019*"wallpaper" + 0.016*"video" + 0.015*"play" + 0.015*"version" + 0.015*"meet" + 0.009*"new" + 0.008*"podcast" + 0.008*"music" + 0.007*"download"')
(2, '0.038*"add" + 0.029*"feature" + 0.028*"option" + 0.027*"map" + 0.013*"search" + 0.011*"keyboard" + 0.009*"file" + 0.008*"useful" + 0.008*"create" + 0.008*"place"')
(3, '0.020*"fix" + 0.019*"try" + 0.018*"tv" + 0.016*"problem" + 0.015*"screen" + 0.015*"issue" + 0.014*"task" + 0.014*"open" + 0.014*"doesnt" + 0.012*"connect"')
(4, '0.028*"nice" + 0.027*"photo" + 0.025*"art" + 0.016*"picture" + 0.012*"best" + 0.011*"selfie" + 0.011*"thank" + 0.011*"easy" + 0.010*"mode" + 0.010*"ok"')


In [28]:
print('Tfidf-Vectorizer: num_topic= {}'.format(num_topic))
lda = models.LdaModel(corpus=corpus2_3, id2word=id2word2_3, num_topics=num_topic, passes=5)
for i in range(len(lda.print_topics())):
    print(lda.print_topics()[i])

Tfidf-Vectorizer: num_topic= 5
(0, '0.017*"crash" + 0.015*"application" + 0.015*"version" + 0.014*"selfie" + 0.011*"doesnt" + 0.011*"device" + 0.010*"screen" + 0.009*"mode" + 0.009*"data" + 0.009*"pc"')
(1, '0.039*"art" + 0.020*"ok" + 0.019*"feature" + 0.018*"meet" + 0.014*"problem" + 0.013*"video" + 0.011*"lot" + 0.010*"class" + 0.010*"watch" + 0.010*"fun"')
(2, '0.032*"wallpaper" + 0.028*"tv" + 0.015*"ad" + 0.011*"measure" + 0.010*"vr" + 0.009*"culture" + 0.008*"world" + 0.008*"service" + 0.008*"point" + 0.007*"interface"')
(3, '0.016*"headphone" + 0.012*"measurement" + 0.012*"option" + 0.012*"sound" + 0.012*"volume" + 0.011*"screen" + 0.010*"button" + 0.010*"download" + 0.009*"thing" + 0.009*"paint"')
(4, '0.022*"map" + 0.020*"photo" + 0.015*"task" + 0.012*"picture" + 0.011*"thank" + 0.010*"apps" + 0.008*"experience" + 0.008*"page" + 0.008*"location" + 0.008*"way"')


-  Count- Vectorizer

- TfidfVectorizer